# Purpose

### 2022-08-05
The CLD3 language codes are not the same as the georelevant codes, so we need a table to make it easier to investigate detected language.

In this table I also
- flag whether a language is included in USE-multilingual (the model used for initial round of topic models)
- Group languages that are not common at reddit as `"Other_language"`

Note that this table was first created in 2021-12, but I needed to update it to add a new column to identify language that (for some reason) is using a numeric ID instead of a code string in the cld3 tables.


# Imports & notebook setup

In [1]:
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload 2

In [2]:
# colab auth for BigQuery & google drive
from google.colab import auth, files, drive
import sys  # need sys for mounting gdrive path

auth.authenticate_user()
print('Authenticated')

Authenticated


### attach up my drive + install my code

In [3]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
g_drive_root = '/content/drive'

try:
    drive._mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive.mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install libraries

In [4]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 10.2 MB/s 
     |████████████████████████████████| 14.2 MB 76.1 MB/s 
     |████████████████████████████████| 965 kB 74.1 MB/s 
     |████████████████████████████████| 144 kB 67.2 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 285 kB 50.3 MB/s 
     |████████████████████████████████| 13.2 MB 68.2 MB/s 
     |████████████████████████████████| 79.9 MB 121 kB/s 
     |████████████████████████████████| 141 kB 93.7 MB/s 
     |████████████████████████████████| 715 kB 69.9 MB/s 
     |████████████████████████████████| 112 kB 81.0 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 181 kB 69.6 MB/s 
     |████████████████████████████████| 146 kB 53.4 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 1.1 MB 61.1 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |█████████████

## General imports

In [33]:
# Regular Imports
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted


# Set env variable needed by some libraries to get datay from BigQuery
# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

### `subclu` imports (custom module)

In [36]:
# subclu imports

# For reloading, need to force-delete some imported items
try:
    del LoadPosts, LoadSubreddits
    del (
        L_CLD3_CODES_FOR_TOP_LANGUAGES_USED_AT_REDDIT,
        L_CLD3_CODES_FOR_TOP_LANGUAGES_AND_USE_MULTILINGUAL,
        D_CLD3_CODE_TO_LANGUAGE_NAME,
    )
except Exception:
    pass

from subclu.utils.hydra_config_loader import LoadHydraConfig
# from subclu.data.data_loaders import LoadPosts, LoadSubreddits
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.utils.language_code_mapping import (
    DF_LANGUAGE_MAPPING,
    L_CLD3_CODES_FOR_TOP_LANGUAGES_USED_AT_REDDIT,
    L_CLD3_CODES_FOR_TOP_LANGUAGES_AND_USE_MULTILINGUAL,
    D_CLD3_CODE_TO_LANGUAGE_NAME,
)

setup_logging()
print_lib_versions([pd, np])

python		v 3.7.13
===
pandas		v: 1.3.5
numpy		v: 1.21.6


# Load df with codes & languages

- `language_name` = default name
- `language_name_top_only` = only show names of top ~40 languages and group other languages as `Other_language`.

Reference article (external analysis) on comments for on week:
- https://towardsdatascience.com/the-most-popular-languages-on-reddit-analyzed-with-snowflake-and-a-java-udtf-4e58c8ba473c

In [37]:
DF_LANGUAGE_MAPPING.shape

(114, 5)

In [38]:
counts_describe(DF_LANGUAGE_MAPPING)

,dtype,count,unique,unique-percent,null-count,null-percent
language_code,object,114,114,100.00%,0,0.00%
language_name,object,114,105,92.11%,0,0.00%
language_name_top_only,object,114,94,82.46%,0,0.00%
language_in_use_multilingual,bool,114,2,1.75%,0,0.00%
language_id,int64,114,108,94.74%,0,0.00%


In [39]:
DF_LANGUAGE_MAPPING.head()

,language_code,language_name,language_name_top_only,language_in_use_multilingual,language_id
5,af,Afrikaans,Afrikaans,False,1
91,sq,Albanian,Albanian,False,87
6,am,Amharic,Other_language,False,2
7,ar,Arabic,Arabic,True,3
43,hy,Armenian,Other_language,False,39


### There should only be 16* languages for USE-multilingual

*15 when we collapse Chinese (standard) + Chinese (traditional) + Chinese (Taiwan)

We see 20 rows because some codes map to a single language.

In [40]:
value_counts_and_pcts(DF_LANGUAGE_MAPPING['language_in_use_multilingual'])

,language_in_use_multilingual-count,language_in_use_multilingual-percent,language_in_use_multilingual-pct_cumulative_sum
False,94,82.5%,82.5%
True,20,17.5%,100.0%


In [42]:
value_counts_and_pcts(
    DF_LANGUAGE_MAPPING[DF_LANGUAGE_MAPPING['language_in_use_multilingual'] == True],
    ['language_in_use_multilingual', 'language_name_top_only'],
    # sort_index=True,
    cumsum=False, top_n=None,
)

In [43]:
DF_LANGUAGE_MAPPING[DF_LANGUAGE_MAPPING['language_in_use_multilingual'] == True]

,language_code,language_name,language_name_top_only,language_in_use_multilingual,language_id
7,ar,Arabic,Arabic,True,3
110,zh-cn,Chinese,Chinese,True,-1
112,zh-tw,Chinese,Chinese,True,-1
109,zh,Chinese,Chinese,True,105
111,zh-Latn,Chinese,Chinese,True,106
74,nl,Dutch,Dutch,True,70
21,en,English,English,True,17
29,fr,French,French,True,25
18,de,German,German,True,14
47,it,Italian,Italian,True,43


# Save table to BigQuery

NOTE: Sorting is not guaranteed in the final BigQuery table.

We can "force" sorting if we set `chunksize` to a number smaller than the full df size.

In [44]:
(
    DF_LANGUAGE_MAPPING
    .assign(table_creation_date=pd.to_datetime(datetime.utcnow().date()))
    .to_gbq(
        destination_table='david_bermejo.language_detection_code_to_name_lookup_cld3',
        project_id='reddit-employee-datasets',
        chunksize=20,
        if_exists='replace'
    )
)

20 out of 114 rows loaded."
40 out of 114 rows loaded."
60 out of 114 rows loaded."
80 out of 114 rows loaded."
100 out of 114 rows loaded."
114 out of 114 rows loaded."
6it [00:24,  4.05s/it]
